# LTL-f BASED-TRACE ALIGNMENT

From constraint to LTL-f: http://www.diag.uniroma1.it/degiacom/papers/2014/AAAI14.pdf

From LTL-f to DFA: http://ltlf2dfa.diag.uniroma1.it/

From LTL-f to automaton: https://github.com/whitemech/logaut

LTL2DFA library: https://github.com/whitemech/LTLf2DFA/

In [1]:
import re
from typing import Match, cast #, Tuple

from ltlf2dfa.parser.ltlf import LTLfParser
from logaut.backends.common.process_mona_output import (
    parse_automaton,
    parse_mona_output,
)

'''from logaut import ltl2dfa
from pylogics.parsers import parse_ltl'''

'from logaut import ltl2dfa\nfrom pylogics.parsers import parse_ltl'

## Constraint automaton

**CONSTRAINTS**

- [x] Chain precedence activity 16 - 17
- [x] Existence activity 1
- [x] Precedence activity 9 -10
- [x] Responded existence activity 5 - 6
- [x] Chain response activity 14 - 15
- [x] Not co-existence activity 19 -20
- [x] Not succession activity 20 -21
- [x] Not chain succession activity 22 - 23
- [x] Response activity 11 - 12
- [x] Absence2 activity 2


In [2]:
def postprocess_output(output: str) -> str:
    """
    Post-process MONA output.
    Capture the output related to the MONA DFA transitions.
    :param: the raw output from the LTLf2DFA tool.
    :return: the output associated to the DFA.
    """
    regex = re.compile(
        r".*(?=\nFormula is (valid|unsatisfiable)|A counter-example)",
        flags=re.MULTILINE | re.DOTALL,
    )
    match = regex.search(output)
    if match is None:
        raise Exception("cannot find automaton description in MONA output.")
    return cast(Match, regex.search(output)).group(0)

In [3]:
parser = LTLfParser()

constraint_formulas = {"existence":"F(a)", 
                       "absence2":"!(F((b & X(F(b)))))", 
                       "response":"G(k -> F(l))", 
                       "precedence":"((!(j) U i) | G(!(j)))",
                       "chain_response":"G((n -> X(o)))", 
                       "chain_precedence": "(!(q) & G((X(q) -> p)))",
                       "responded_existence":"(F(e) -> F(f))", 
                       "not_coexistence":"!((F(s) & F(t)))", 
                       "not_succession":"G((t -> !(F(u))))", 
                       "not_chain_succession":"G((v <-> !(X(w))))"}

'\nfor name,f in constraint_formulas.items():\n    formula = parser(f)       # returns an LTLfFormula\n    print(formula)\n\n    dfa = formula.to_dfa(mona_dfa_out=True)\n    dfa_file = open("automata/"+name+".txt", "w")\n    dfa_file.write(dfa)\n    dfa_file.close()\n\n    mona_output = parse_mona_output(postprocess_output(dfa))\n    automaton = parse_automaton(mona_output)\n    automaton.to_graphviz().render("automata/"+name+".dfa", view=True)\n'

In [34]:
constraint_formulas = {"existence":"F(a)","response":"G(k -> F(l))"}
index = 1

all_automata = {}

for type_constr,f in constraint_formulas.items():
    build_automaton = {}
    build_automaton["name"] = type_constr
    build_automaton["formula"] = f

    all_states_constr = []
    final_states_constr = []
    init_states_constr = []
    rejecting_states_constr = []
    rho_constr_basic = []

    #FARE TUTTO

    ## Parser + dfa #####################################################
    formula = parser(f)       # returns an LTLfFormula
    #print(formula)
    dfa = formula.to_dfa(mona_dfa_out=True)
    mona_output = parse_mona_output(postprocess_output(dfa))
    automaton = parse_automaton(mona_output)
    automaton.to_graphviz().render("automata/"+type_constr+".dfa", view=True)

    ## for DELLA CELLA DOPO #############################################
    #...

    # FINE


    build_automaton["all_states"] = all_states_constr
    build_automaton["final_states"] = final_states_constr
    build_automaton["init_states"] = init_states_constr
    build_automaton["rejecting_states"] = rejecting_states_constr
    build_automaton["transitions"] = rho_constr_basic

    all_automata["a"+str(index)] = build_automaton

    index += 1

In [ ]:
for type_constr,_ in constraint_formulas.items():
    all_states_constr[type_constr] = []
    final_states_constr[type_constr] = []
    rejecting_states_constr[type_constr] = []
    check_elem = []
    rho_constr_basic[type_constr] = []

    dfa_file = open("automata/"+type_constr+".txt", "r")
    for line in dfa_file:
        if ("DFA for formula with free variables" in line):
            start_index = line.find(':') + 2
            for c in range(start_index, len(line), 2):
                if (line[c] != '\n'):
                    check_elem.append(line[c].lower())
                    if (line[c].lower() not in symbols_constr):
                        symbols_constr.append(line[c].lower())

        if ("Initial state" in line):
            start_index = line.find(':') + 2
            for c in range(start_index, len(line), 2):
                if (line[c] != '\n'):
                    init_states_constr[type_constr] = "s"+str(int(line[c])+index_states)

        if ("Accepting states" in line):
            start_index = line.find(':') + 2
            for c in range(start_index, len(line), 2):
                if (line[c] != '\n'):
                    final_states_constr[type_constr].append("s"+str(int(line[c])+index_states))
                    all_states_constr[type_constr].append("s"+str(int(line[c])+index_states))
        if ("Rejecting states" in line):
            start_index = line.find(':') + 2
            for c in range(start_index, len(line), 2):
                if (line[c] != '\n'):
                    all_states_constr[type_constr].append("s"+str(int(line[c])+index_states))
                    rejecting_states_constr[type_constr].append("s"+str(int(line[c])+index_states))
        
        if ("State" in line):
            start_index = line.find(':') -1
            s_start = "s"+str(int(line[start_index])+index_states)
            s_end = "s"+str(int(line[-2])+index_states)
            if (len(check_elem) == 1):
                value = line[start_index+3]
                for elem in symbols_constr:
                    if (value == 'X'):
                        rho_constr_basic[type_constr].append(s_start+" "+elem+" "+s_end)
                    if (value == '0'):
                        if (elem != check_elem[0]):
                            rho_constr_basic[type_constr].append(s_start+" "+elem+" "+s_end)
                    if (value == '1'):
                        if (elem == check_elem[0]):
                            rho_constr_basic[type_constr].append(s_start+" "+elem+" "+s_end)
            if (len(check_elem) == 2):
                value1 = line[start_index+3]
                value2 = line[start_index+4]
                value = value1+value2
                for elem in symbols_constr:
                    if (value == 'XX'):
                        rho_constr_basic[type_constr].append(s_start+" "+elem+" "+s_end)
                    elif (value == '0X'):
                        if (elem != check_elem[0]):
                            rho_constr_basic[type_constr].append(s_start+" "+elem+" "+s_end)
                    elif (value == 'X0'):
                        if (elem != check_elem[1]):
                            rho_constr_basic[type_constr].append(s_start+" "+elem+" "+s_end)
                    elif (value == '1X' or value == '10'):
                        if (elem == check_elem[0]):
                            rho_constr_basic[type_constr].append(s_start+" "+elem+" "+s_end)
                    elif (value == 'X1' or value == '01'):
                        if (elem == check_elem[1]):
                            rho_constr_basic[type_constr].append(s_start+" "+elem+" "+s_end)
                    elif (value == '00'):
                        if (elem != check_elem[0] and elem != check_elem[1]):
                            rho_constr_basic[type_constr].append(s_start+" "+elem+" "+s_end)
        
    index_states += len(all_states_constr[type_constr])


    print (check_elem)
    print (all_states_constr)
    print (final_states_constr)
    print (init_states_constr)
    print (rejecting_states_constr)
    print (symbols_constr)
    print ("____")
    print (index_states)
    print (rho_constr_basic[type_constr])
    print ("___________________________")

## Load xes file
containing the trace

In [ ]:
log_path = "dataset/logs/synthetic-logs/10constraints/1-constraint-inverted/log-from-10constr-model-1constr_inverted-1-50.xes"

In [ ]:
alphabet = "abcdefghijklmnopqrstuvwxyz"
voc = {}
for i in range(len(alphabet)):
    voc[i+1] = alphabet[i]

def convertNumberToChar (val):
    return voc[val]

In [ ]:
# Legge dal file .xes, estrapola le tracce, trasforma il valore delle tracce in caratteri,
# infine aggiungile al log sottoforma di stringhe
def readLog (log_path):
    # Inizializzazione variabili
    flag = False;           # Indica quando dobbiamo leggere un evento dal file
    trace = [];             # Lista di eventi sottoforma di interi
    traceChar = [];         # Lista di eventi sottoforma di char
    traceString = "";       # Stringa composta da eventi sottoforma di char
    log = []               # Lista di tracce ognuna delle quali è una stringa di char

    # Apriamo il file e leggiamolo riga per riga
    f = open  (log_path)
    f1 = f.readlines()
    
    # Per ogni riga del file...
    for x in f1:
        # Se c'è un evento, attiviamo la flag
        if (x.__contains__("<event>")):
            flag = True
        # Se flag attiva e siamo sulla riga dove è presente il nome dell'evento,
        # estrapoliamo il nome dell'evento e appendiamolo a trace
        if (flag and x.__contains__('<string key="concept:name"')):
            val = x.split('value="activity ',1)[1]
            val = val.split('"')[0]                
            trace.append(val)
            flag = False
        # Quando non ci sono più eventi possiamo lavorare sulla traccia in questione
        if (x.__contains__("</trace>")):
            for event in trace:
                traceChar.append(convertNumberToChar(int(event)))  # Converti gli eventi in char 
            traceString = "".join(traceChar)                       # Lista di eventi -> stringa
            log.append(traceString)                                # Appendi stringa a log

            # Inizializza nuovamente le variabili
            trace = []
            traceChar = []
            traceString = ""
    return log

In [ ]:
log = readLog(log_path) # list of traces

#trace = log[2]
#trace = 'stkai'
#print (trace)

## Trace automaton

In [ ]:
symbols_trace = list(set(trace))

print(trace)
print(symbols_trace)

In [ ]:
rho_trace_basic = []           # rho = [[q0, w, q1],[q1,k,q2],...,[..,.,qn-1]]
Q_trace = []
for i in range(len(trace)):
    Q_trace.append('t'+str(i))
    rho_trace_basic.append('t'+str(i)+" "+trace[i] +" "+ 't'+str(i+1))

Q_trace.append('t'+str(len(trace)))
init_state_trace = Q_trace[0]
final_state_trace = Q_trace[-1]

print(Q_trace)
print(rho_trace_basic)

## Planning domain D

In [ ]:
domain_name = "domain_multi"
problem_name = "problem_multi"

In [ ]:
pddl_domain_initial = "(define (domain "+domain_name+") "\
                "(:requirements :strips :typing :action-costs) "\
                "(:types trace_state automaton_state - state activity) "

pddl_domain_predicates = "(:predicates (trace ?t1 - trace_state "\
                            "?e - activity "\
                            "?t2 - trace_state) "\
                            "(automaton ?s1 - automaton_state "\
                            "?e - activity "\
                            "?s2 - automaton_state) "\
                            "(cur_state ?s - state) "\
                            "(final_state ?s - state)) "


pddl_domain_actions =  "(:action sync "\
                        ":parameters (?t1 - trace_state ?e - activity ?t2 - trace_state) "\
                        ":precondition (and (cur_state ?t1) (trace ?t1 ?e ?t2)) "\
                        ":effect(and (not (cur_state ?t1)) (cur_state ?t2) "\
                        "(forall (?s1 ?s2 - automaton_state) "\
                        "(when (and (cur_state ?s1) "\
                        "(automaton ?s1 ?e ?s2)) "\
                        "(and (not (cur_state ?s1)) "\
                        "(cur_state ?s2)))))) "\
                        "(:action add "\
                        ":parameters (?e - activity) "\
                        ":effect (and (increase (total-cost) 1) "\
                        "(forall (?s1 ?s2 - automaton_state) "\
                        "(when (and (cur_state ?s1) "\
                        "(automaton ?s1 ?e ?s2)) "\
                        "(and (not (cur_state ?s1)) "\
                        "(cur_state ?s2)))))) "\
                        "(:action del "\
                        ":parameters (?t1 - trace_state ?e - activity "\
                        "?t2 - trace_state) "\
                        ":precondition (and (cur_state ?t1) (trace ?t1 ?e ?t2)) "\
                        ":effect (and (increase (total-cost) 1) "\
                        "(not (cur_state ?t1)) (cur_state ?t2))))"

pddl_domain = pddl_domain_initial + pddl_domain_predicates + pddl_domain_actions

print (pddl_domain)

## PDDL problem

In [ ]:
symbols = symbols_constr.copy()
'''print (Q_trace)
print (all_states_constr)
print (symbols)
print (rho_trace_basic)
print (rho_constr_basic)'''

In [ ]:
#type_constraints = ['existence', 'absence2', 'response', 'precedence', 'chain_response','responded_existence', 'chain_precedence', 'not_coexistence', 'not_succession', 'not_chain_succession']
type_constraints = ["existence","response"]

pddl_problem_initial = "(define (problem "+problem_name+") (:domain "+domain_name+") "
pddl_problem_objects = "(:objects "

for q in Q_trace:
    pddl_problem_objects += q+" "
pddl_problem_objects += "- trace_state "

for type_constr in type_constraints:
    for q in all_states_constr[type_constr]:
        pddl_problem_objects += q+" "
    pddl_problem_objects += "- automaton_state "

for s in symbols:
    pddl_problem_objects += s+" "
pddl_problem_objects += "- activity"

pddl_problem_objects += ") "


pddl_problem_init = "(:init (= (total-cost) 0) (cur_state "+init_state_trace+") "
for trace in rho_trace_basic:
    pddl_problem_init += "(trace "+trace+") "
pddl_problem_init += "(final_state "+final_state_trace+") "

for type_constr in type_constraints:
    pddl_problem_init += "(cur_state "+init_states_constr[type_constr]+") " 
    for trace in rho_constr_basic[type_constr]:
        pddl_problem_init += "(automaton "+trace+") "

    for i in range(len(final_states_constr[type_constr])):
        pddl_problem_init += "(final_state "+final_states_constr[type_constr][i]+") "
pddl_problem_init += ") "


pddl_problem_goal = "(:goal (forall (?s - state) "\
                    "(imply (cur_state ?s) (final_state ?s)))) "
pddl_problem_metric = "(:metric minimize (total-cost)))"
pddl_problem = pddl_problem_initial + pddl_problem_objects + pddl_problem_init + pddl_problem_goal + pddl_problem_metric
print (pddl_problem)

## Save .pddl files

In [ ]:
file1 = open("PDDL/"+domain_name+".pddl", "w")
file1.write(pddl_domain)
file1.close()

file2 = open("PDDL/"+problem_name+".pddl", "w")
file2.write(pddl_problem)
file2.close()

Fast-downward planner:
`./fast-downward.py domain_multi.pddl problem_multi.pddl --search "astar(blind())"`